# Skillbox ML Junior. Общий файл с поиском моделей и исследованиями.

### XGB: Тестовый AUC: 0.7586 (случайный лес не рассматриваем в поиске по сетке с начальным 0.71, т.к. XGB - более продвинутый вариант леса)
### NN: Тестовый AUC: 0.7532


## 1. Загружаем библиотеки, определяем колонки


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [4]:
import xgboost as xgb
from xgboost import XGBClassifier
import copy
import dill

In [7]:
print(xgb.__version__)

3.0.5


In [9]:
# смотрим периодически за объемом памяти
import psutil
print(f"Доступно RAM: {psutil.virtual_memory().available / (1024**3):.2f} GB")

Доступно RAM: 6.79 GB


In [11]:
# периодически удаляем из памяти ненужные файлы на определенном этапе
import gc #сборщик мусора (garbage collector)
#del large_variable
#gc.collect()

In [13]:
from pathlib import Path
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils.class_weight import compute_class_weight

In [15]:
# для воспроизводимости один генератор чисел везде
RAND_SEED = 42

columnsOK = ['id', 'rn', 'pre_pterm', 'pre_fterm', 'pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_outstanding',
             'pre_loans_total_overdue', 'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate',
             'pre_loans5', 'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90',
             'is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90',
             'pre_util', 
             'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit',
             'enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9',
             'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18',
             'enc_paym_19', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24',
             'enc_loans_account_holder_type', 'enc_loans_credit_status', 'enc_loans_credit_type', 'enc_loans_account_cur',
             'pclose_flag', 'fclose_flag', 'pre_till_pclose', 'pre_till_fclose', 'pre_since_opened', 'pre_since_confirmed']

## 2. Функции для записи файла с созданием фичей до агрегации, агрегацией и созданием фичей после агрегации

In [31]:
### 1. **FeatureGeneratorBeforeAgg** — трансформер для создания фичей до агрегации
def fast_mode(series):
    modes = series.mode()
    return modes.iloc[0] if not modes.empty else 0  # ← возвращаем 0, а не np.nan

class FeatureGeneratorBeforeAgg(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        # ВРЕМЕННЫЕ: время жизни кредита и доля времени
        X['loan_age'] = X['pre_fterm'] - X['pre_since_opened']
        # ФИНАНСОВЫЕ: общая нагрузка по кредитам
        X['total_outstanding'] = X.groupby('id')['pre_loans_outstanding'].transform('sum')
        # ПРОСРОЧКИ: добавляем веса
        X['weighted_late_score'] = (
            X['pre_loans5'] * 1 +
            X['pre_loans530'] * 2 +
            X['pre_loans3060'] * 3 +
            X['pre_loans6090'] * 4 +
            X['pre_loans90'] * 5
        )
        return X

### 2. **Aggregator** — трансформер для агрегации по `id`
class Aggregator(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ohe_cat_cols_ = None  # ← сохраним список OHE-столбцов при fit      
    def fit(self, X, y=None):
        X = X.copy()
        # определения столбцов
        ohe_cols = ['enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9',
                    'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18',
                    'enc_paym_19', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24',
                    'enc_loans_account_holder_type', 'enc_loans_credit_status', 'enc_loans_credit_type', 'enc_loans_account_cur',
                    'pre_util', 'pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_credit_cost_rate']
        # Проверяем, какие столбцы есть
        existing_ohe_cols = [col for col in ohe_cols if col in X.columns]
        if not existing_ohe_cols:
            raise KeyError(f"Нет столбцов для OHE при fit: {ohe_cols}")

        # Применяем get_dummies
        X = pd.get_dummies(X, columns=existing_ohe_cols, prefix=existing_ohe_cols)
        # Сохраняем список OHE-столбцов
        self.ohe_cat_cols_ = [col for col in X.columns if col.startswith(tuple([f"{c}_" for c in existing_ohe_cols]))]
        
        return self

    def transform(self, X):
        X = X.copy()
        # Определение ohe_cols (должно совпадать с fit)
        # print("Столбцы после FeatureGeneratorBeforeAgg:", X.columns.tolist())
        ohe_cols = ['enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9',
                    'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18',
                    'enc_paym_19', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24',
                    'enc_loans_account_holder_type', 'enc_loans_credit_status', 'enc_loans_credit_type', 'enc_loans_account_cur',
                    'pre_util', 'pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_credit_cost_rate']
 
        # Проверяем, какие столбцы есть
        existing_ohe_cols = [col for col in ohe_cols if col in X.columns]
        if not existing_ohe_cols:
            raise KeyError(f"Нет столбцов для OHE при transform: {ohe_cols}")

        # Применяем get_dummies
        X = pd.get_dummies(X, columns=existing_ohe_cols, prefix=existing_ohe_cols)

        # Восстанавливаем недостающие OHE-столбцы (те, что были при fit)
        for col in self.ohe_cat_cols_:
            if col not in X.columns:
                X[col] = 0        
    
        # АГРЕГИРУЕМ С УЧЕТОМ ПРЕДВАРИТЕЛЬНЫХ ФИЧ 
        new_cols = ['total_outstanding']
        new_weight_cols = ['weighted_late_score']
        new_age_cols = ['loan_age']
        max_cols = ['rn']  
        date_cols = ['pre_pterm', 'pre_fterm', 'pre_till_pclose', 'pre_till_fclose', 'pre_since_opened', 'pre_since_confirmed']
        abs_sum_cols1 = ['pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_credit_cost_rate']
        ratio_cols = ['pre_util']
        cat_cols = ['enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9',
                     'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18',
                     'enc_paym_19', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24',
                     'enc_loans_account_holder_type', 'enc_loans_credit_status',
                     'enc_loans_credit_type', 'enc_loans_account_cur']
        num_overdues = ['pre_loans5', 'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90']
        abs_sum_cols2 = ['pre_loans_outstanding', 'pre_loans_total_overdue', 'pre_loans_max_overdue_sum']
        flag_cols = ['is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90',
                     'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit']
        date_flag_cols = ['pclose_flag', 'fclose_flag']

        # Используем self.ohe_cat_cols_, которые были сохранены при fit
        ohe_cat_cols = self.ohe_cat_cols_

        # Определяем агрегационные правила
        agg_rules = {}
        for col in max_cols:
            agg_rules[col] = 'max'
        for col in ohe_cat_cols:
            agg_rules[col] = 'sum'
        for col in num_overdues:
            agg_rules[col] = fast_mode
        for col in abs_sum_cols2:
            agg_rules[col] = fast_mode
        for col in flag_cols:
            agg_rules[col] = 'mean'
        for col in date_flag_cols:
            agg_rules[col] = 'mean'
        for col in new_cols:
            agg_rules[col] = 'max'
        for col in new_weight_cols:
            agg_rules[col] = 'sum'
        for col in new_age_cols:
            agg_rules[col] = 'median'
        for col in date_cols:
            agg_rules[col] = 'median'

        # Агрегируем
        df_agg = X.groupby('id').agg(agg_rules).reset_index()

# Понижаем размерность, но исключаем столбец 'id'
        for col in df_agg.select_dtypes(include=['int64', 'int32', 'int16']).columns:
            if col != 'id':  # ← исключаем 'id'
                df_agg[col] = df_agg[col].astype('float32')                        # был конфликт с imputer, меняем 16 на 32 'float32'
        for col in df_agg.select_dtypes(include=['float64']).columns:
            df_agg[col] = df_agg[col].astype('float32')

        return df_agg

### 3. **FeatureCreatorAfterAgg** — трансформер для создания фичей после агрегации
class FeatureCreatorAfterAgg(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        # Отличный клиент
        X['is_perfect'] = (
            (X['is_zero_loans5'] == 1) &
            (X['is_zero_loans530'] == 1) &
            (X['is_zero_loans3060'] == 1) &
            (X['is_zero_loans6090'] == 1) &
            (X['is_zero_loans90'] == 1) &
            (X['is_zero_util'] == 1)
        ).astype('int8')
        # Оценка платежей
        for month in range(0, 25):
            cols = [f'enc_paym_{month}_{i}' for i in range(5) if f'enc_paym_{month}_{i}' in X.columns]
            if cols:
                weights = list(range(len(cols)))
                X[f'paym_score_{month}'] = sum(X[col] * w for col, w in zip(cols, weights))
        # Общая оценка платежей
        paym_cols = [f'paym_score_{m}' for m in range(25) if f'paym_score_{m}' in X.columns]
        X['avg_payment_score'] = X[paym_cols].mean(axis=1)
        X['recent_payment_score'] = X[['paym_score_0', 'paym_score_1', 'paym_score_2']].mean(axis=1)
        X['payment_trend'] = X['paym_score_0'] - X['paym_score_12']
        credit_type_cols = [col for col in X.columns if col.startswith('enc_loans_credit_type_')]
        X['num_credit_types'] = X[credit_type_cols].sum(axis=1)
        # Интенсивность кредитования
        X['loans_per_time'] = X['rn'] / (X['pre_since_opened'] + 1)
        return X

# функция только ЧТЕНИЕ Читаем подмножество Parquet-файлов из локальной директории
def read_parquet_dataset_from_local(
    path_to_dataset: str,
    start_from: int = 0,
    num_parts_to_read: int = 1,
    columns=columnsOK,
    verbose: bool = False
) -> pd.DataFrame:
    dataset_dir = Path(path_to_dataset)
    if not dataset_dir.exists():
        raise FileNotFoundError(f"Директория не найдена: {dataset_dir}")

    # Ищем только .parquet файлы, начинающиеся с 'train'
    dataset_paths = sorted([
        f for f in dataset_dir.iterdir()
        if f.is_file() and f.name.startswith('train') and f.suffix.lower() in ['.parquet', '.pq']
    ])

    if not dataset_paths:
        raise ValueError(f"Не найдено подходящих файлов в {dataset_dir}. "
                         "Ожидаются файлы вида 'train*.parquet' или 'train*.pq'.")

    if verbose:
        print(f"Найдено файлов: {len(dataset_paths)}")
        print("Список файлов:", [p.name for p in dataset_paths])

    start_from = max(0, start_from)
    end_index = start_from + num_parts_to_read
    chunks = dataset_paths[start_from:end_index]

    if not chunks:
        raise ValueError("Нет файлов для чтения с заданными параметрами "
                         f"(start_from={start_from}, num_parts_to_read={num_parts_to_read}).")

    if verbose:
        print(f"\nЧтение {len(chunks)} файлов:")
        for p in chunks:
            print(f"  - {p}")

    res = []
    for chunk_path in tqdm(chunks, desc="Чтение Parquet-файлов", disable=not verbose):
        chunk = pd.read_parquet(chunk_path, columns=columns)
        res.append(chunk)

    combined_df = pd.concat(res, ignore_index=True)

    if verbose:
        print(f"\nЗагружено строк: {len(combined_df)}")
        if columns:
            print(f"Загружены колонки: {list(combined_df.columns)}")

    return combined_df

# ПОДГОТОВКА И ЧТЕНИЕ
def prepare_transactions_dataset(
    path_to_dataset: str,
    num_parts_total: int = 50,
    num_parts_to_preprocess_at_once: int = 1,
    save_to_path: str | None = None,
    verbose: bool = False,
    preprocess_fn=None,
    columns_to_load=None
) -> pd.DataFrame | None:
    input_dir = Path(path_to_dataset)
    if not input_dir.exists():
        raise FileNotFoundError(f"Директория с данными не найдена: {input_dir}")

    if save_to_path:
        output_dir = Path(save_to_path)
        output_dir.mkdir(parents=True, exist_ok=True)

    all_processed_frames = []

    steps = range(0, num_parts_total, num_parts_to_preprocess_at_once)
    for step in tqdm(steps, desc="Обработка транзакций", disable=not verbose):
        df = read_parquet_dataset_from_local(
            path_to_dataset=str(input_dir),
            start_from=step,
            num_parts_to_read=num_parts_to_preprocess_at_once,
            columns=columns_to_load,
            verbose=False
        )

        if preprocess_fn is not None:
            df = preprocess_fn(df)
        else:
            df = df.drop_duplicates().reset_index(drop=True)

        if save_to_path:
            chunk_name = f"processed_chunk_{step:03d}.pq"
            df.to_parquet(output_dir / chunk_name, index=False)

        all_processed_frames.append(df)

    if not all_processed_frames:
        raise RuntimeError("Ни один блок данных не был обработан.")

    final_df = pd.concat(all_processed_frames, ignore_index=True)

    if verbose:
        print(f"\n✅ Итоговый датафрейм: {final_df.shape[0]:,} строк, {final_df.shape[1]} колонок.")

    return final_df

In [12]:
# смотрим периодически за объемом памяти
print(f"Доступно RAM: {psutil.virtual_memory().available / (1024**3):.2f} GB")

Доступно RAM: 9.75 GB


## 3. Пайплайн подготовки данных (создание фич и агрегации) и запись файла с новыми фичами

In [13]:
# СТАРТ
# 1. Функции подготовки данных
def my_preprocess(df):
    df = df.copy()
    feature_pipeline = Pipeline([
        ('generate_features_before_agg', FeatureGeneratorBeforeAgg()),
        ('aggregate', Aggregator()),
        ('create_features_after_agg', FeatureCreatorAfterAgg()),
    ])
    X_agg = feature_pipeline.fit_transform(df)
    return X_agg

# 2. Запуск чтения
df = prepare_transactions_dataset(
    path_to_dataset='D:/train_data/',
    num_parts_total=12,
    num_parts_to_preprocess_at_once=2,
    save_to_path='D:/TEST/',
    verbose=True,
    preprocess_fn=my_preprocess,
    columns_to_load=columnsOK
)
# проверка размерности)
print("Всего строк:", len(df))
print("Уникальных id:", df['id'].nunique())
print("Дубликаты по id:", len(df) - df['id'].nunique())

Обработка транзакций: 100%|█████████████████████████████████████████████████████████████| 6/6 [32:34<00:00, 325.75s/it]



✅ Итоговый датафрейм: 3,000,000 строк, 247 колонок.
Всего строк: 3000000
Уникальных id: 3000000
Дубликаты по id: 0


In [14]:
df.to_parquet('D:/TEST/transactions.pq', index=False) #чтобы сохранить типы данных запишем в parquet

### НЕ ЗАПУСКАТЬ!!!, т.к. далее для проверки модели просто читаем датафрейм и проверяем его

In [59]:
# ПРОВЕРКА - прочитаем получившийся файл
df_r = pd.read_parquet('D:/TEST/transactions.pq')
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Columns: 247 entries, id to enc_loans_credit_type_7
dtypes: float32(245), int64(1), int8(1)
memory usage: 2.8 GB


In [61]:
with pd.option_context('display.max_columns', None):
    display(df_r.tail(10))

,id,rn,enc_paym_0_0,enc_paym_0_1,enc_paym_0_2,enc_paym_0_3,enc_paym_1_0,enc_paym_1_1,enc_paym_1_2,enc_paym_1_3,enc_paym_2_0,enc_paym_2_1,enc_paym_2_2,enc_paym_2_3,enc_paym_3_0,enc_paym_3_1,enc_paym_3_2,enc_paym_3_3,enc_paym_4_0,enc_paym_4_1,enc_paym_4_2,enc_paym_4_3,enc_paym_5_0,enc_paym_5_1,enc_paym_5_2,enc_paym_5_3,enc_paym_6_0,enc_paym_6_1,enc_paym_6_2,enc_paym_6_3,enc_paym_7_0,enc_paym_7_1,enc_paym_7_2,enc_paym_7_3,enc_paym_8_0,enc_paym_8_1,enc_paym_8_2,enc_paym_8_3,enc_paym_9_0,enc_paym_9_1,enc_paym_9_2,enc_paym_9_3,enc_paym_10_0,enc_paym_10_1,enc_paym_10_2,enc_paym_10_3,enc_paym_11_1,enc_paym_11_2,enc_paym_11_3,enc_paym_11_4,enc_paym_12_0,enc_paym_12_1,enc_paym_12_2,enc_paym_12_3,enc_paym_13_0,enc_paym_13_1,enc_paym_13_2,enc_paym_13_3,enc_paym_14_0,enc_paym_14_1,enc_paym_14_2,enc_paym_14_3,enc_paym_15_0,enc_paym_15_1,enc_paym_15_2,enc_paym_15_3,enc_paym_16_0,enc_paym_16_1,enc_paym_16_2,enc_paym_16_3,enc_paym_17_0,enc_paym_17_1,enc_paym_17_2,enc_paym_17_3,enc_paym_18_0,enc_paym_18_1,enc_paym_18_2,enc_paym_18_3,enc_paym_19_0,enc_paym_19_1,enc_paym_19_2,enc_paym_19_3,enc_paym_20_1,enc_paym_20_2,enc_paym_20_3,enc_paym_20_4,enc_paym_21_0,enc_paym_21_1,enc_paym_21_2,enc_paym_21_3,enc_paym_22_0,enc_paym_22_1,enc_paym_22_2,enc_paym_22_3,enc_paym_23_0,enc_paym_23_1,enc_paym_23_2,enc_paym_23_3,enc_paym_24_1,enc_paym_24_2,enc_paym_24_3,enc_paym_24_4,enc_loans_account_holder_type_0,enc_loans_account_holder_type_1,enc_loans_account_holder_type_2,enc_loans_account_holder_type_3,enc_loans_account_holder_type_4,enc_loans_account_holder_type_5,enc_loans_account_holder_type_6,enc_loans_credit_status_0,enc_loans_credit_status_1,enc_loans_credit_status_2,enc_loans_credit_status_3,enc_loans_credit_status_4,enc_loans_credit_status_5,enc_loans_credit_status_6,enc_loans_credit_type_0,enc_loans_credit_type_1,enc_loans_credit_type_2,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pre_util_0,pre_util_1,pre_util_2,pre_util_3,pre_util_4,pre_util_5,pre_util_6,pre_util_7,pre_util_8,pre_util_9,pre_util_10,pre_util_11,pre_util_12,pre_util_13,pre_util_14,pre_util_15,pre_util_16,pre_util_17,pre_util_18,pre_util_19,pre_loans_credit_limit_0,pre_loans_credit_limit_1,pre_loans_credit_limit_2,pre_loans_credit_limit_3,pre_loans_credit_limit_4,pre_loans_credit_limit_5,pre_loans_credit_limit_6,pre_loans_credit_limit_7,pre_loans_credit_limit_8,pre_loans_credit_limit_9,pre_loans_credit_limit_10,pre_loans_credit_limit_11,pre_loans_credit_limit_12,pre_loans_credit_limit_13,pre_loans_credit_limit_14,pre_loans_credit_limit_15,pre_loans_credit_limit_16,pre_loans_credit_limit_17,pre_loans_credit_limit_18,pre_loans_credit_limit_19,pre_loans_next_pay_summ_0,pre_loans_next_pay_summ_1,pre_loans_next_pay_summ_2,pre_loans_next_pay_summ_3,pre_loans_next_pay_summ_4,pre_loans_next_pay_summ_5,pre_loans_next_pay_summ_6,pre_loans_credit_cost_rate_0,pre_loans_credit_cost_rate_1,pre_loans_credit_cost_rate_2,pre_loans_credit_cost_rate_3,pre_loans_credit_cost_rate_4,pre_loans_credit_cost_rate_5,pre_loans_credit_cost_rate_6,pre_loans_credit_cost_rate_7,pre_loans_credit_cost_rate_8,pre_loans_credit_cost_rate_9,pre_loans_credit_cost_rate_10,pre_loans_credit_cost_rate_11,pre_loans_credit_cost_rate_12,pre_loans_credit_cost_rate_13,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,total_outstanding,weighted_late_score,loan_age,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_since_opened,pre_since_confirmed,is_perfect,paym_score_0,paym_score_1,paym_score_2,paym_score_3,paym_score_4,paym_score_5,paym_score_6,paym_score_7,paym_score_8,paym_score_9,paym_score_10,paym_score_11,paym_score_12,paym_score_13,paym_score_14,paym_score

In [63]:
print("Всего строк:", len(df_r))
print("Уникальных id:", df_r['id'].nunique())
print("Минимальный id:", df_r['id'].min())
print("Максимальный id:", df_r['id'].max())
print("Минимальный индекс:", df_r.index.min())
print("Максимальный индекс:", df_r.index.max())

Всего строк: 3000000
Уникальных id: 3000000
Минимальный id: 0
Максимальный id: 2999999
Минимальный индекс: 0
Максимальный индекс: 2999999


In [65]:
df_r[df_r['id']==2999998]

,id,rn,enc_paym_0_0,enc_paym_0_1,enc_paym_0_2,enc_paym_0_3,enc_paym_1_0,enc_paym_1_1,enc_paym_1_2,enc_paym_1_3,...,paym_score_22,paym_score_23,paym_score_24,avg_payment_score,recent_payment_score,payment_trend,num_credit_types,loans_per_time,enc_loans_credit_type_6,enc_loans_credit_type_7
999998,2999998,5.0,4.0,0.0,0.0,1.0,4.0,0.0,0.0,1.0,...,15.0,15.0,15.0,12.68,4.0,-12.0,5.0,0.5,0.0,0.0


In [67]:
del df_r
gc.collect()

229

In [71]:
# смотрим периодически за объемом памяти
print(f"Доступно RAM: {psutil.virtual_memory().available / (1024**3):.2f} GB")

Доступно RAM: 7.50 GB


## 4. Основной пайплайн, обучение модели и запись моделей в pickle

In [29]:
new_cols = ['total_outstanding']
new_weight_cols = ['weighted_late_score']
new_age_cols = ['loan_age']
max_cols = ['rn']  
date_cols = ['pre_pterm', 'pre_fterm', 'pre_till_pclose', 'pre_till_fclose', 'pre_since_opened', 'pre_since_confirmed']
abs_sum_cols1 = ['pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_credit_cost_rate']
ratio_cols = ['pre_util']
cat_cols = ['enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9',
             'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18',
             'enc_paym_19', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24',
             'enc_loans_account_holder_type', 'enc_loans_credit_status',
             'enc_loans_credit_type', 'enc_loans_account_cur']
num_overdues = ['pre_loans5', 'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90']
abs_sum_cols2 = ['pre_loans_outstanding', 'pre_loans_total_overdue', 'pre_loans_max_overdue_sum']
flag_cols = ['is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90',
             'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit']
date_flag_cols = ['pclose_flag', 'fclose_flag']

ohe_cols = cat_cols + ratio_cols + abs_sum_cols1

feature_pipeline = Pipeline([
    ('generate_features_before_agg', FeatureGeneratorBeforeAgg()),
    ('aggregate', Aggregator()),
    ('create_features_after_agg', FeatureCreatorAfterAgg()),
])

In [31]:
# 3. Читаем данные и получаем датафрейм с таргетами
df_targets = pd.read_csv('D:/TEST/train_target.csv')

# 4. Теперь объединяем с target по id
X_final = pd.merge(df, df_targets[['id', 'flag']], on='id', how='left')

# Проверяем, что все target не NaN
if X_final['flag'].isnull().any():
    print("⚠️ Внимание: некоторые клиенты не имеют target!")
    X_final = X_final.dropna(subset=['flag'])
    
print(f'Размер датафрейма: {X_final.shape}')
print(f'Нет пропусков: {X_final.isna().sum().max() == 0}')    
# проверка размерности)
print("Всего строк:", len(X_final))
print("Уникальных id:", X_final['id'].nunique())
print("Дубликаты по id:", len(X_final) - X_final['id'].nunique())

# 5. Выделяем X и y для обучения
X_train = X_final.drop(columns=['id', 'flag'])
y_train = X_final['flag']

# 6. Препроцессор
ohe_cat_cols = [col for col in df.columns if col.startswith(tuple([f"{c}_" for c in ohe_cols]))]

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

ct = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', MinMaxScaler())
        ]), new_cols + new_age_cols + max_cols + date_cols + num_overdues + abs_sum_cols2),
        ('cat', SimpleImputer(strategy='constant', fill_value=0), ohe_cat_cols), # выделил отдельно!!!!!!!!
        ('passthrough', 'passthrough', flag_cols + date_flag_cols)
    ],
    remainder='drop'
)

# 7. Обучение модели
best_params = {
    'colsample_bytree': 0.8916028672163949,
    'gamma': 0.38563517334297287,
    'learning_rate': 0.03221339552022711,
    'max_depth': 5,
    'min_child_weight': 9,
    'reg_alpha': 2.4646297296191846,
    'reg_lambda': 2.846065804843518,
    'subsample': 0.7797802696552814,
    'n_estimators': 3000
}

pipeline_full = Pipeline([
    ('preprocessor', ct),
    ('classifier', XGBClassifier(
        **best_params,
        random_state=RAND_SEED,
        eval_metric='auc',
        use_label_encoder=False,
        tree_method='hist',
        scale_pos_weight=class_weights[1],
        verbosity=0
    ))
])

pipeline_full.fit(X_train, y_train)

Размер датафрейма: (3000000, 248)
Нет пропусков: False
Всего строк: 3000000
Уникальных id: 3000000
Дубликаты по id: 0


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['total_outstanding',
                                                   'loan_age', 'rn',
                                                   'pre_pterm', 'pre_fterm',
                                                   'pre_till_pclose',
                                                   'pre_till_fclose',
                                                   'pre_since_opened',
                                                   'pre_since_confirmed',
                                                   'pre_loans5', 'pre_loans530',
                                                   'pre_loans3060',
                                                   'pre_loans6090',
                                                   'pre_loan...
                               gamma=0.38563517334297287, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.03221339552022711, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=5,
                               max_leaves=None, min_child_weight=9, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=3000, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [32]:
# После обучения записываем в pickle с помощью dill чтобы сохранить классы
with open('feature_pipeline.pkl', 'wb') as f:
    dill.dump(feature_pipeline, f)  # если feature_pipeline нужен вне цикла
with open('full_model.pkl', 'wb') as f:
    dill.dump(pipeline_full, f)



## 5. Проверка работы модели на выборочных данных из исходных паркетов.

In [17]:
# смотрим периодически за объемом памяти
print(f"Доступно RAM: {psutil.virtual_memory().available / (1024**3):.2f} GB")

Доступно RAM: 6.84 GB


In [19]:
# Загружаем feature_pipeline и full_model
with open('feature_pipeline.pkl', 'rb') as f:
    feature_pipeline = dill.load(f)
with open('full_model.pkl', 'rb') as f:
    full_model = dill.load(f)


In [21]:
# Новые данные используем как сырые данные одного из паркетов с неполным количеством столбцов
df_new = pd.read_parquet('D:/train_data/train_data_4.pq') #можем менять паркеты для проверки
df_targ = pd.read_csv('D:/TEST/train_target.csv')
print(f'Размер таргета: {df_targ.shape}')
print(f'Размер исходного: {df_new.shape}')

Размер таргета: (3000000, 2)
Размер исходного: (2064110, 61)


In [33]:
# Новые данные
# Выберем 10 000 уникальных id (можно изменить)
n_ids = 100000
unique_ids = df_new['id'].drop_duplicates().sample(n=n_ids, random_state=RAND_SEED).values
print(f"Выбрано {len(unique_ids)} уникальных id")

# Отфильтруем df_new по этим id
df_subset = df_new[df_new['id'].isin(unique_ids)].copy()
print(f"Размер подмножества: {df_subset.shape}")

# Оставим только те id, для которых есть target
ids_with_target = df_targ['id'].values
df_subset = df_subset[df_subset['id'].isin(ids_with_target)]
print(f"После фильтрации по target: {df_subset.shape}")

#X_agg = feature_pipeline.transform(df_subset) #разобъем для контроля на три шага!!!!!1

# Шаг 1: FeatureGeneratorBeforeAgg
step1 = feature_pipeline.named_steps['generate_features_before_agg']
try:
    X1 = step1.transform(df_subset)
    print("✅ Шаг 1 успешен. Shape:", X1.shape)
except Exception as e:
    print("❌ Ошибка в шаге 1 (FeatureGeneratorBeforeAgg):", e)
    raise

# Шаг 2: Aggregator
step2 = feature_pipeline.named_steps['aggregate']
try:
    X2 = step2.transform(X1)
    print("✅ Шаг 2 успешен. Shape:", X2.shape)
except Exception as e:
    print("❌ Ошибка в шаге 2 (Aggregator):", e)
    raise

# Шаг 3: FeatureCreatorAfterAgg
step3 = feature_pipeline.named_steps['create_features_after_agg']
try:
    X3 = step3.transform(X2)
    print("✅ Шаг 3 успешен. Shape:", X3.shape)
except Exception as e:
    print("❌ Ошибка в шаге 3 (FeatureCreatorAfterAgg):", e)
    raise

# Присоединяем target
df_with_target = pd.merge(
    X3, #X_agg,
    df_targ[['id', 'flag']],
    on='id',
    how='inner'
)

X_test = df_with_target.drop(columns=['id', 'flag'])
y_test = df_with_target['flag']

# 2. Получить список признаков, на которых обучалась модель
# Если full_model — Pipeline, и внутри есть preprocessor
preprocessor = full_model.named_steps['preprocessor']  # или другое имя
expected_features = full_model.feature_names_in_
# 3. Добавить недостающие колонки
for col in expected_features:
    if col not in X_test.columns:
        X_test[col] = 0

print(f"X_test shape: {X_test.shape}, y_test: {y_test.shape}")

preds = full_model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, preds)
print(f"🎯 AUC на подмножестве ({n_ids} id): {auc:.4f}")

Выбрано 100000 уникальных id
Размер подмножества: (827976, 61)
После фильтрации по target: (827976, 61)
✅ Шаг 1 успешен. Shape: (827976, 64)
✅ Шаг 2 успешен. Shape: (100000, 208)
✅ Шаг 3 успешен. Shape: (100000, 239)
X_test shape: (100000, 246), y_test: (100000,)
🎯 AUC на подмножестве (100000 id): 0.7972


### при ошибках обучать агрегатор или запускать функции

In [25]:
# Проверьте, что ohe_cat_cols_ не None
agg_step = feature_pipeline.named_steps['aggregate']
print("ohe_cat_cols_ до трансформации:", agg_step.ohe_cat_cols_)

ohe_cat_cols_ до трансформации: None


In [27]:
# чтобы не выдавало ошибку❌ Ошибка в шаге 2 (Aggregator):'NoneType' object is not iterable Обучите Aggregator на подмножестве df_new (до трансформации)
# Или запускать ячейку функций выше!!!!!!!!!!!!!!
agg_step = feature_pipeline.named_steps['aggregate']

# Возьмите небольшое подмножество, чтобы обучить
sample_ids = df_new['id'].drop_duplicates().sample(n=1000, random_state=42).values
df_sample = df_new[df_new['id'].isin(sample_ids)]

# Обучаем Aggregator
agg_step.fit(df_sample)

print("ohe_cat_cols_ после fit:", agg_step.ohe_cat_cols_)

ohe_cat_cols_ после fit: ['enc_paym_0_0', 'enc_paym_0_1', 'enc_paym_0_2', 'enc_paym_0_3', 'enc_paym_1_0', 'enc_paym_1_1', 'enc_paym_1_2', 'enc_paym_1_3', 'enc_paym_2_0', 'enc_paym_2_1', 'enc_paym_2_2', 'enc_paym_2_3', 'enc_paym_3_0', 'enc_paym_3_1', 'enc_paym_3_2', 'enc_paym_3_3', 'enc_paym_4_0', 'enc_paym_4_1', 'enc_paym_4_2', 'enc_paym_4_3', 'enc_paym_5_0', 'enc_paym_5_1', 'enc_paym_5_2', 'enc_paym_5_3', 'enc_paym_6_0', 'enc_paym_6_1', 'enc_paym_6_2', 'enc_paym_6_3', 'enc_paym_7_0', 'enc_paym_7_1', 'enc_paym_7_2', 'enc_paym_7_3', 'enc_paym_8_0', 'enc_paym_8_1', 'enc_paym_8_2', 'enc_paym_8_3', 'enc_paym_9_0', 'enc_paym_9_1', 'enc_paym_9_2', 'enc_paym_9_3', 'enc_paym_10_0', 'enc_paym_10_1', 'enc_paym_10_2', 'enc_paym_10_3', 'enc_paym_11_1', 'enc_paym_11_2', 'enc_paym_11_3', 'enc_paym_11_4', 'enc_paym_12_0', 'enc_paym_12_1', 'enc_paym_12_2', 'enc_paym_12_3', 'enc_paym_13_0', 'enc_paym_13_1', 'enc_paym_13_2', 'enc_paym_13_3', 'enc_paym_14_0', 'enc_paym_14_1', 'enc_paym_14_2', 'enc_paym_1

Выбрано 10000 уникальных id
Размер подмножества: (81773, 61)
После фильтрации по target: (81773, 61)
✅ Шаг 1 успешен. Shape: (81773, 64)
✅ Шаг 2 успешен. Shape: (10000, 209)
✅ Шаг 3 успешен. Shape: (10000, 240)
X_test shape: (10000, 246), y_test: (10000,)
🎯 AUC на подмножестве (10000 id): 0.8101

## 6. Дополнительные материалы

### проверки и оптимизации размеров

In [43]:
# Проверьте, что ohe_cat_cols_ не None
agg_step = feature_pipeline.named_steps['aggregate']
print("ohe_cat_cols_:", agg_step.ohe_cat_cols_)

ohe_cat_cols_: None


In [86]:
print(feature_pipeline)
print(len(expected_features))

Pipeline(steps=[('generate_features_before_agg', FeatureGeneratorBeforeAgg()),
                ('aggregate', Aggregator()),
                ('create_features_after_agg', FeatureCreatorAfterAgg())])
214


In [88]:
print(df_new.shape)
print(df_new.info())

(2150908, 275)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2150908 entries, 0 to 2150907
Columns: 275 entries, id to passthrough__fclose_flag
dtypes: int64(275)
memory usage: 4.4 GB
None
Pipeline(steps=[('generate_features_before_agg', FeatureGeneratorBeforeAgg()),
                ('aggregate', Aggregator()),
                ('create_features_after_agg', FeatureCreatorAfterAgg())])


In [ ]:
# убрали сразу две колонки  'pre_over2limit', 'pre_maxover2limit',
columnsOK = ['id', 'rn', 'pre_pterm', 'pre_fterm', 'pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_outstanding',
             'pre_loans_total_overdue', 'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate',
             'pre_loans5', 'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90',
             'is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90',
             'pre_util', 
             'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit',
             'enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9',
             'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18',
             'enc_paym_19', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24',
             'enc_loans_account_holder_type', 'enc_loans_credit_status', 'enc_loans_credit_type', 'enc_loans_account_cur',
             'pclose_flag', 'fclose_flag', 'pre_till_pclose', 'pre_till_fclose', 'pre_since_opened', 'pre_since_confirmed']

In [ ]:
# более надежно для продакшна
import joblib
joblib.dump(pipeline, 'pipeline.joblib')

In [ ]:
model_info = {
    'pipeline': pipeline,
    'feature_names': feature_names,
    'model_version': '1.0',
    'training_date': '2024-01-01'
}

joblib.dump(model_info, 'model_package.joblib')

In [ ]:
# ОБЪЯСНЕНИЕ:     Создаем трансформер
feature_creator = FeatureCreator()
# fit - "обучаем" (ничего не делаем)
feature_creator.fit(X_train)
# transform - применяем преобразования
X_train_with_new_features = feature_creator.transform(X_train)
# Или одной операцией
X_train_with_new_features = feature_creator.fit_transform(X_train)

In [ ]:
pipeline = Pipeline([
    ('feature_creator', FeatureCreator()),  # Этот шаг
    ('preprocessor', ct),
    ('classifier', XGBClassifier())
])

# При pipeline.fit():
# 1. feature_creator.fit(X_train) → возвращает self
# 2. feature_creator.transform(X_train) → данные с новыми фичами
# 3. preprocessor.fit(данные_с_фичами)
# 4. preprocessor.transform(данные_с_фичами)
# 5. classifier.fit(преобразованные_данные, y_train)

# При pipeline.predict(new_data):
# 1. feature_creator.transform(new_data) → автоматически, без fit!
# 2. preprocessor.transform(данные_с_фичами)
# 3. classifier.predict(преобразованные_данные)

In [19]:
# Найдём столбцы с типом bool
bool_cols = df.select_dtypes(include='bool').columns
# Преобразуем их в int8 
df[bool_cols] = df[bool_cols].astype('int8')

In [24]:
# Проверку на inf также проводим (возможно сразу исправить в файле по чтению данных)
inf_cols = []
for col in df.columns:
    if df[col].dtype in ['float32', 'float64']:
        if np.isinf(df[col]).any():
            inf_cols.append(col)
            print(f"Колонка {col} содержит inf")

print("Колонки с inf:", inf_cols)

Колонки с inf: []


In [21]:
# при наличии object
for col in object_cols:
    # Приведём к нижнему регистру и заменим 'true'/'false' на булевы значения
    if df[col].dropna().isin(['True', 'False', 'true', 'false']).all():
        df[col] = df[col].str.capitalize().map({'True': 1, 'False': 0}).astype('int8')
    else:
        # Если не логические строки — попробуем привести к числу (осторожно!)
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int8')

In [20]:
# Найдем столбцы типа object
object_cols = df.select_dtypes(include='object').columns
object_cols

Index([], dtype='object')

In [22]:
# Заполняем ВСЕ пропущенные значения нулями В ИСХОДНОМ ДАТАФРЕЙМЕ СРАЗУ
df.fillna(0, inplace=True)

In [23]:
df.isna().sum().max() == 0

True

### еще доп фичи

In [ ]:
#strategy='most_frequent'

In [ ]:
#Интерактивные фичи (feature interactions)
# Просрочка × тип кредита
df['overdue_x_type6'] = df['pre_loans_total_overdue'] * df['enc_loans_credit_type_6']
# Платёжный риск × статус
df['payment_x_status'] = df['avg_payment_score'] * df['bad_status_score']

In [ ]:
df['overdue_x_util'] = df['pre_loans_total_overdue'] * df['pre_util']
df['type6_x_overdue'] = df['enc_loans_credit_type_6'] * df['pre_loans_total_overdue']

In [ ]:
# Интенсивность кредитования ВРЕМЕННЫЕ
#df['loans_per_time'] = df['rn'] / (df['pre_since_opened'] + 1)  # кредитов на день истории

# Доля закрытых кредитов
df['closed_loans_ratio'] = (df['pre_fterm'] > 0).astype(int) / df['rn']

# Флаг: активный кредит с просрочкой
df['active_overdue'] = ((df['pre_till_fclose'] > 0) & (df['pre_loans_total_overdue'] > 0)).astype(int)

In [ ]:
# Комбинированный риск
df['combined_risk_score'] = (
    df['pre_util'] * 0.3 +
    df['pre_over2limit'] * 0.4 +
    df['pre_maxover2limit'] * 0.3
)

# Отношение просрочки к остатку
df['overdue_to_outstanding'] = df['pre_loans_total_overdue'] / (df['pre_loans_outstanding'] + 1e-6)

# Нормализованный овердрафт
df['overdue_ratio'] = df['pre_loans_total_overdue'] / (df['pre_loans_credit_limit'] + 1e-6)

In [ ]:
#credit_type_cols = [col for col in df.columns if col.startswith('enc_loans_credit_type_')]
#df['num_credit_types'] = df[credit_type_cols].sum(axis=1)  # сколько типов кредитов у клиента

# Если типы упорядочены по риску (например, 6 = микрозайм), можно сделать индекс:
risk_weights = {0: 0, 1: 0, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 3}  # задайте по бизнес-логике
df['credit_type_risk_index'] = sum(
    df[f'enc_loans_credit_type_{i}'] * risk_weights.get(i, 0)
    for i in range(8)
)